# Forma canônica de observador e observabilidade

Tal como no caso do regulador de estados, há uma forma de espaço de estados para a qual a solução do observador é muito simples. A forma é conhecida como forma canônica de observador.

Para deduzi-la procedemos como anteriormente. Desenhamos um diagrama de blocos e extraimos as equações de estados dos integradores.

O caso de terceira ordem geral é mostrado na Figura abaixo.

<p align="center">
<img src="Fig7.31.svg" width="80%">
</p>

A função de transferência correspondente é:
$$
\begin{align*}
    G(s) = \frac{b_1s^2+b_2s+b_3}{s^3+a_1s^2+a_2s+a_3}
\end{align*}
$$

Note que:
* Os integradores **não** estão em série diretamente
* Entre cada integrador há um somador
* Apenas a saída (último integrador) é realimentada 
* A realimentação é feita para cada um dos somadores, através dos ganhos do denominador
* A entrada se conecta a cada um dos somadores, através dos ganhos do numerador

A representação de estados é:

$$
\begin{align*}
    \mathbf{\dot{x}} &= \left[
        \begin{array}{rrr}
        -a_1 & 1 & 0\\
        -a_2 & 0 & 1\\
        -a_3 & 0 & 0\end{array}
    \right]\mathbf{{x}}+
    \left[\begin{array}{rrr}
        b_1\\
        b_2\\
        b_3\end{array}
    \right]u\\
    y &= \left[\begin{array}{ccc} 1 & 0 & 0\end{array}\right]\mathbf{x}
\end{align*}
$$

Note que:
* Na matriz $\mathbf{F}$, a primeira coluna é formada pelos coeficientes do denominador com sinal trocado, ordem crescente de potência de $s$, de cima para baixo. 
* As colunas restantes podem ser montadas usando uma matriz identidade de ordem 2 e uma linha de zeros
* A matriz $\mathbf{G}$ é uma coluna formada pelos coeficientes do numerador, ordem crescente de potência de $s$ de cima para baixo.
* A matriz $\mathbf{H}$ é uma linha de zeros, exceto pelo primeiro elemento igual a 1.
* $J=0$.

## Exercício:
Desenhe e obtenha as matrizes para o caso de 4a ordem.


A forma de observador é útil no projeto de observadores. Para o caso de 3a ordem anterior, a matriz de projeto é
$$
\begin{align*}
\mathbf{F-LH} &= \left[
        \begin{array}{rrr}
        -a_1-l_1 & 1 & 0\\
        -a_2-l_2 & 0 & 1\\
        -a_3-l_3 & 0 & 0\end{array}
    \right]
\end{align*}
$$
cuja equação característica é:
$$
\begin{align*}
s^3+(a_1+l_1)s^2+(a_2+l_2)s+(a_3+l_3)=0
\end{align*}
$$

Assim, podemos achar os ganhos do observador $\mathbf{L}=\left[\begin{array}{ccc} l_1 & l_2 & l_3\end{array} \right]$ muito facilmente com o polinômio desejado.

## Observabilidade

Tal como no caso do regulador, a forma observável e a capacidade de encontrar um observador de estados é uma propriedade intrínseca do sistema, e que independen da representação de estados usada.

Essa propriedade se chama observabilidade. 

Observabilidade é a capacidade que um sistema possui em "permitir" que seus estados sejam estimados a partir apenas do conhecimento do sinal de saída.

Da mesma forma que a controlabilidade, podemos pedir a observabilidade pela **matriz de observabilidade** e seu determinante.

A matriz de observabilidade é construída linha por linha, como:
$$
\begin{align*}
\mathbf{\mathcal{O}} &= \left[
        \begin{array}{c}
        \mathbf{H}\\
        \mathbf{HF}\\
        \mathbf{HF^2}\\
        \vdots\\
        \mathbf{HF^{n-1}}
        \end{array}
    \right]
\end{align*}
$$

Uma forma rápida de calcular essa matriz é usar a função obsv() da biblioteca *control*.


Um sistema SISO é observável se $\det \mathbf{\mathcal{O}} \neq 0$. Se o sistema é MIMO, devemos olhar para o posto da matriz de observabilidade. 

O sistema perde observabilidade quando há cancelamentos entre pólos zeros, de forma semalhante à controlabilidade. O sistema torna-se mais observável à medida que possui mais saídas mensuráveis.

Observe que existem diversos paralelos entre controlabilidade e observabilidade, 
inclusive nos cálculos. 

Como já mostramos, os cálculos de observabilidades basicamente trocam a matriz $\mathbf{G}$ pela transposta de $\mathbf{H}$ e a matriz de estados pela sua transposta. Esse "paralelismo" é chamado de *dualidade* entre as duas propriedades. 

In [1]:
import control as ct
import numpy as np

In [17]:
a = np.array([[5,0,-1,0]])
b = [0,9,0,-8]
#F = np.array([[],[],[],[]])
M = np.block([[np.eye(3)],[np.zeros((1,3))]])
F = np.block([-a.T,M])
H = np.array([[1,0,0,0]])

In [18]:
O = ct.obsv(F,H)
O

array([[   1.,    0.,    0.,    0.],
       [  -5.,    1.,    0.,    0.],
       [  25.,   -5.,    1.,    0.],
       [-124.,   25.,   -5.,    1.]])

In [19]:
O1 = np.block([[H],[H@F],[H@(F@F)],[H@(F@F@F)]])
O1

array([[   1.,    0.,    0.,    0.],
       [  -5.,    1.,    0.,    0.],
       [  25.,   -5.,    1.,    0.],
       [-124.,   25.,   -5.,    1.]])

In [20]:
print(np.linalg.det(O))
print(np.linalg.matrix_rank(O))

1.0000000000000002
4


In [42]:
num = 10*np.convolve([1,1],[1,2])
den = np.convolve([1,0],np.convolve([1,5],[1,0.99999999]))
G = ct.tf(num,den)
print(G)


10 s^2 + 30 s + 20
------------------
s^3 + 6 s^2 + 5 s



In [43]:
sys = ct.ss(G)
print(sys)

<LinearIOSystem>: sys[21]
Inputs (1): ['u[0]']
Outputs (1): ['y[0]']
States (2): ['x[0]', 'x[1]']

A = [[-5.00000000e+00 -8.88178420e-16]
     [-1.00000000e+00 -1.11022302e-16]]

B = [[10.]
     [ 0.]]

C = [[ 1. -2.]]

D = [[0.]]



In [47]:
np.roots([-0.0416,0.2496])

array([6.])

In [44]:
O = ct.obsv(sys.A,sys.C)

In [45]:
print(np.linalg.matrix_rank(O))
print(np.linalg.det(O))

2
-5.999999999999992
